In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating summaries

In [3]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split, get_sample_weights, get_eval_set
from src.preprocessing import preprocess_data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, AutoModelForCausalLM
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from collections import defaultdict
import transformers
from peft import get_peft_model, LoraConfig, TaskType
import re
from bert_score import BERTScorer
import langid
from src.utils import aggregate_samples, evaluate_model, compute_class_weights, remove_hashtag_links, get_first_texts
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BitsAndBytesConfig

tqdm.pandas()

In [4]:
train_data, test_data = train_test_split()


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:04<00:00,  3.50it/s]


In [5]:
df = pd.concat(train_data)

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data')
df['tokens'] = df['Tweet'].progress_apply(tokenizer.tokenize)

target_words = [
    "goal", "penalty", "halftime", "full-time", "yellow", "red",
    "kickoff", "extra time", "stoppage time", "foul", "offside", "handball",
    "save", "tackle", "dribble", "corner", "substitution", "header",
    "free kick", "throw-in", "assist", "hat-trick", "own goal", "victory",
    "defeat", "draw", "win", "loss", "tie", "comeback", "goalkeeper",
    "striker", "midfielder", "defender", "referee", "fans", "var", "gooal"
]
target_words = set(tokenizer.tokenize(" ".join(target_words)))

def is_valid_text(t):
    for w in t:
        if w in target_words:
            return True
        
    return False

df['is_valid']= df['tokens'].progress_apply(is_valid_text)
# df['lan'] = df['Tweet'].progress_apply(lambda x : langid.classify(x)[0])

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/users/eleves-a/2022/pedro.silva/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 1472980/1472980 [00:01<00:00, 1130731.72it/s]


In [6]:
en_df = df.query("is_valid == 1")#.query("lan == 'en' ")

In [7]:
en_df

ID  MatchID  PeriodID  EventType      Timestamp  \
0  14         0_0        0         0          0  1403725802000   
   29         0_0        0         0          0  1403725806000   
   36         0_0        0         0          0  1403725807000   
   41         0_0        0         0          0  1403725809000   
   76         0_0        0         0          0  1403725818000   
...           ...      ...       ...        ...            ...   
19 155326  19_129       19       129          1  1403560797000   
   155330  19_129       19       129          1  1403560798000   
   155448  19_129       19       129          1  1403560799000   
   155458  19_129       19       129          1  1403560799000   
   155507  19_129       19       129          1  1403560800000   

                                                       Tweet  \
0  14      Let's go Honduras, with a little help from Fra...   
   29      The more goals France scores today the less go...   
   36      Robbie Fowler predicting a Honduras win? #Worl...   
   41         #WorldCup2014 lets go Ecuador lets go Honduras   
   76      I’m following Honduras versus Switzerland in t...   
...                                                      ...   
19 155326  #mirelagh: #WorldCup2014/ #WM2014 / Full Time:...   
   155330            Great win can't wait for next game #MEX   
   155448  Round of 16 - 29 de Junio #MEX vs #NED #Brasil...   
   155458  Holland better not let up, El Tri is coming fo...   
   155507               We took over Twitter today lmao #MEX   

                                                      tokens  is_valid  
0  14      [Let, 's, Ġgo, ĠHonduras, ,, Ġwith, Ġa, Ġlittl...      True  
   29      [The, Ġmore, Ġgoals, ĠFrance, Ġscores, Ġtoday,...      True  
   36      [Rob, bie, ĠFowler, Ġpredicting, Ġa, ĠHonduras...      True  
   41      [#, World, C, up, 2014, Ġlets, Ġgo, ĠEcuador, ...      True  
   76      [I, âĢ, Ļ, m, Ġfollowing, ĠHonduras, Ġversus, ...      True  
...                                                      ...       ...  
19 155326  [#, mire, l, agh, :, Ġ#, World, C, up, 2014, /...      True  
   155330  [Great, Ġwin, Ġcan, 't, Ġwait, Ġfor, Ġnext, Ġg...      True  
   155448  [Round, Ġof, Ġ16, Ġ-, Ġ29, Ġde, ĠJun, io, Ġ#, ...      True  
   155458  [H, oll, and, Ġbetter, Ġnot, Ġlet, Ġup, ,, ĠEl...      True  
   155507  [We, Ġtook, Ġover, ĠTwitter, Ġtoday, Ġl, ma, o...      True  

[476071 rows x 8 columns]

In [21]:
possible_indices = set(train_data.keys())

test_indices = list(np.random.choice(list(possible_indices), size=3, replace = False,))
test_indices = [13,1,18]
all_train_indices = list(possible_indices.difference(set(test_indices)))
val_indices = [1,5,12,19]
# val_indices = list(np.random.choice(all_train_indices, 3, replace=False))
# train_indices = list(set(all_train_indices).difference(set(val_indices)))
# train_indices = [0,2,7,11,13,18]



train_df = aggregate_samples(en_df, list(possible_indices), max_size = 10)
# train_df = get_samples(train_indices, df = train)
test_df = aggregate_samples(en_df, test_indices, max_size=10)
val_df = aggregate_samples(en_df, val_indices, max_size=10)

/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7fa95c6320c0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7fa95c6320c0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7fa95c6320c0> is currently using SeriesGroupBy.mean.

In [22]:
train_df = remove_hashtag_links(train_df)
test_df = remove_hashtag_links(test_df)
val_df = remove_hashtag_links(val_df)

In [23]:
test_indices

[13, 1, 18]

## Training the model

In [24]:
train_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.539541
0.0    0.460459
Name: proportion, dtype: float64

In [25]:
val_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.558522
0.0    0.441478
Name: proportion, dtype: float64

In [26]:
base_prompt = '''


You are an AI tasked with analyzing a collection of tweets posted during a single minute of a football match. Your goal is to generate a concise summary of the key events that occurred during this time and to specifically answer whether any of the following events occurred: 

1. A goal (including who scored, if mentioned).
2. A yellow or red card (including the player or team, if mentioned).
3. A kickoff (start of a half).
4. Halftime or fulltime whistle.

{tweets}

### Instructions:
1. Analyze the tweets for clear indications of the above events using common football-related terminology, phrases, or hashtags.
2. If the event is ambiguous or not explicitly stated in the tweets, mark it as "Not mentioned."
3. Summarize any additional significant match events or fan reactions from the tweets that are relevant to understanding the minute.
4. Return the response in the following structured format:

```
**Summary of Events:**
[Your summary here.]

**Specific Event Analysis:**
- Goal: [Yes/No/Not mentioned] [Additional details if yes.]
- Yellow Card: [Yes/No/Not mentioned] [Additional details if yes.]
- Red Card: [Yes/No/Not mentioned] [Additional details if yes.]
- Kickoff: [Yes/No/Not mentioned] [Additional details if yes.]
- Halftime: [Yes/No/Not mentioned]
- Fulltime: [Yes/No/Not mentioned]
```

Use this structure to organize the analysis clearly and concisely.

'''

In [31]:
train_df['Tweet'].apply(lambda x: len(x.split(" "))).sort_values()

MatchID  PeriodID
2        76            1
0        70            1
2        69            1
0        85            1
         92            1
                    ... 
18       54          311
         126         311
17       43          313
3        125         314
         55          320
Name: Tweet, Length: 2137, dtype: int64

In [36]:
train_df

Tweet  \
MatchID PeriodID                                                      
0       0         Robbie Fowler predicting a Honduras win? \n le...   
        1         can  please win...\nLego Honduras and France !...   
        2         Hoy golea Costly. \nMy Predictions: \n 1-3 \n ...   
        3         Honduras better kick some ass\nCome on Hondura...   
        4         Honduras is taking the win lets go!!\nHonduras...   
...                                                             ...   
19      125       What a win!\n \nWe got this  win!!!\n 3-1   \n...   
        126       He'll yea 3-1 \n doing work \nGreat win  \n\n ...   
        127       FULL TIME!!!\n 1-3 \n\nCielito Lindo!  \nGood ...   
        128       Great win for \n for the win\n vs  lets go!\nW...   
        129       gonna win 1-0 against \nMexico will beat Nethe...   

                  EventType   ID  
MatchID PeriodID                  
0       0               0.0    9  
        1               0.0   17  
        2               0.0   17  
        3               0.0   19  
        4               0.0   18  
...                     ...  ...  
19      125             1.0  113  
        126             1.0  106  
        127             1.0   88  
        128             1.0   76  
        129             1.0   59  

[2137 rows x 3 columns]

In [ ]:
# K Fold CV


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    cache_dir = "/Data"    
)

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    quantization_config = quantization_config,
    device_map="auto",
    cache_dir = "/Data" 
)
# tokenizer.pad_token = tokenizer.eos_token


device = 'cuda'

generated_data = []

with torch.no_grad():
    for idx, row in tqdm(train_df.iterrows(), total = len(train_df)):
        text = row['Tweet']
        label = row['EventType']

        match_id, period_id = idx

        prompt = base_prompt.format(tweets = text)
        message = [ {"role": "user", "content": prompt}]

        template = tokenizer.apply_chat_template(
            message,
            tokenize= False
        )

        tokens = tokenizer(
            template,
            return_tensors = 'pt'
        )

        generated_ids = base_model.generate(
            tokens['input_ids'].to('cuda'),
            # attention_mask = tokens['attention_mask'].to("cuda"),
            max_new_tokens = 200,
            do_sample = False,
            # temperature = 1.
        )

        decoded = tokenizer.batch_decode(generated_ids[:, tokens['input_ids'].shape[1]:])
        generated_t = decoded[0].split("<|end_header_id|>")[1].split("<|eot_id|>")[0]

        clear_output()
        print(f'''
            generated text : {generated_t}
            label : {label}
            '''
        )

        data = {
            'summary': generated_t,
            "label": label,
            "Tweet": text,
            "MatchID": match_id,
            "PeriodID":period_id
        }

        generated_data.append(data)
# Combine results for this fold
# validation_results = pd.DataFrame({
#     "MatchID": validation_data["MatchID"].values,
#     "true_values": labels,
#     "predictions": preds,
# })

# final_results.append(validation_results)

  0%|          | 8/2137 [00:44<3:25:30,  5.79s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



            generated text : 

**Summary of Events:**
The tweets suggest that the match is between Honduras and France, and the fans are showing their support for Honduras. The tweets are filled with enthusiastic messages, urging Honduras to win, but there is no indication of any significant events occurring during this minute.

**Specific Event Analysis:**
- Goal: Not mentioned
- Yellow Card: Not mentioned
- Red Card: Not mentioned
- Kickoff: Not mentioned
- Halftime: Not mentioned
- Fulltime: Not mentioned
            label : 0.0
            


  0%|          | 8/2137 [00:45<3:23:30,  5.74s/it]


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(generated_data).to_csv("generated_data.csv")

: 